# Connecting to the Congressional District Geospatial Data

### Import Needed Filepaths and Libraries

In [1]:
import math

# import geopandas to simplify working with geospatial data in the district shapefile
import geopandas as gpd

try:
    from shapely import make_valid
    HAS_MAKE_VALID = True
except Exception:
    HAS_MAKE_VALID = False

# import filepath of congressional district geospatial data from config.py
from texas_gerrymandering_hb4.config import PLANC2308_SHP_FILE, INTERIM_DATA_DIR

2025-09-07 18:17:44.627 | INFO     | texas_gerrymandering_hb4.config:<module>:11 - PROJ_ROOT path is: /home/aimlexpert/Documents/GitHub/texas-gerrymandering-HB4


### Load Congressional Districts Shapefile

In [3]:
# load PLANC2308 Congressional district shapefile
gdf = gpd.read_file(PLANC2308_SHP_FILE)
print(f"Loaded {len(gdf)} rows")
gdf.head()

Loaded 38 rows


,District,geometry
0,1,"POLYGON ((1558608.508 1007368.924, 1558424.339..."
1,2,"POLYGON ((1430214.842 865650.638, 1430191.262 ..."
2,3,"POLYGON ((1494406.98 1201348.426, 1494404.818 ..."
3,4,"POLYGON ((1294445.052 1205704.041, 1294442.732..."
4,5,"POLYGON ((1403804.511 1045105.488, 1403812.104..."


### Fix Invalid Geometries
Shapely handles most topology errors; otherwise buffer(0) is used.

In [ ]:
if HAS_MAKE_VALID:
    gdf["geometry"] = gdf.geometry.apply(make_valid)
else:
    gdf["geometry"] = gdf.buffer(0)
print("Fixed geometry errors (if any)".)

# Converting the Congressional District Shapefile Into a Geopackage File

In [9]:
# save shapefile as a geopackage
gdf.to_file(INTERIM_DATA_DIR/"districts.gpkg", driver="GPKG")

# Initial EDA on District Geospatial Data

In [12]:
# reading the newly created districts.gpkg file
districts = gpd.read_file(INTERIM_DATA_DIR/"districts.gpkg", driver="GPKG")

# viewing the first 5 rows of the dataset
districts.head()

   District                                           geometry
0         1  POLYGON ((1558608.508 1007368.924, 1558424.339...
1         2  POLYGON ((1430214.842 865650.638, 1430191.262 ...
2         3  POLYGON ((1494406.980 1201348.426, 1494404.818...
3         4  POLYGON ((1294445.052 1205704.041, 1294442.732...
4         5  POLYGON ((1403804.511 1045105.488, 1403812.104...


In [14]:
# viewing the last 5 rows of the dataset
districts.tail()

    District                                           geometry
33        34  POLYGON ((1261750.064 412835.286, 1261695.727 ...
34        35  POLYGON ((1216875.807 725701.423, 1216868.212 ...
35        36  POLYGON ((1468189.541 825548.094, 1468131.384 ...
36        37  POLYGON ((1215880.409 884311.182, 1215642.827 ...
37        38  POLYGON ((1433143.468 848921.024, 1433142.046 ...


In [16]:
# viewing dataset information
districts.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   District  38 non-null     int64   
 1   geometry  38 non-null     geometry
dtypes: geometry(1), int64(1)
memory usage: 740.0 bytes


# Converting the Geopackage to a new Coordinate Reference System (CRS)

In [25]:
# Converting coordinate reference system to global web maps
assert gdf.crs.to_epsg() == 4326

# Lowercasing Column Names

In [19]:
# lowercasing all the attributes names in dataset
gdf.columns = gdf.columns.str.lower()

# Checking for Invalid Geometry

In [21]:
invalid_geometry = gdf[~gdf.is_valid]
if not invalid_geometry.empty:
    print("Invalid geometry is found at these indices:")
    print(invalid_geometry.index)
else:
    print("No invalid geometry occurs in the dataset.")

No invalid geometry occurs in the dataset.


# Sorting and Indexing Congressional Districts

In [22]:
# discarding the old index and adopting the default integer index
gdf = gdf.sort_values("district").reset_index(drop="True")

# Adding a Derived Column for the District Area  & Perimeter in Square Kilometers

In [30]:
# Reprojecting geometries  to Texas Centric Albers Equal Area
gdf_texas_area = gdf.to_crs(epsg=3081)

# computing the district area
# Texas Centric Albers Equal Area is in meters, so we must convert to sq km
gdf["area_sq_km"] = gdf_texas_area.geometry.area / 1e6

#computing the district perimeter
# Texas Centric Albers Equal area is in meters, so we must convert to km
gdf["perimeter_km"] = gdf_texas_area.length / 1e3

# Adding a Derived Column for Polbsy-Popper Compactness Score

In [31]:
gdf["polsby-popper"] = 4 * math.pi * gdf_texas_area.area / (gdf_texas_area.length ** 2)